<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/NBs_Google_Colab/DL_S11_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Curso Deep Learning: Economía</h1>

## S11: Deep Neural Networks.

# 1. Problema del desvanecimiento/explosión del gradiente.

Como hemos visto, las redes neuronales son entrenadas usando el algoritmo del **descenso del gradiente estocástico** (hasta ahora) conjuntamente con el de **propagación hacia atrás** (o retropropagación). 

Lo anterior consiste, en primer lugar, en el cálculo del error de la predicción realizada por el modelo y el uso de este error para estimar un gradiente que es usado para actualizar cada peso en la red, de tal forma que se reduzca el error en la siguiente iteración. Este error del gradiente es propagado hacia atrás a lo largo de la red, desde la salida a la capa de entrada.

Normalmente, se desea entrenar la red neuronal con muchas capas, con la intensión de que el incremento del número de capas aumente la capacidad predictiva de la red, haciéndola capaz de aprender un dataset de gran tamaño. Por otra parte, se espera también que al aumentar el número de capas se logre representar de forma eficiente funciones de mapeo más complejas de los datos de entrada a las predicciones a la salida de la red.

Un problema en el entrenamiento de redes con muchas capas, **deep neural networks**, es que el gradiente se **desvanece o explota** dramáticamente a medida que se propaga hacia atrás a lo largo de la red. En el caso del desvanecimiento del gradiente, la actualización del error alcanzado en las capas cercanas a la entrada puede ser tan pequeño que tenga un efecto despreciable. El desvanecimiento del gradiente dificulta saber en qué dirección se deben variar los parámetros para obtener una mejora en la función de coste.

El término del **desvanecimiento/explosión** del gradiente se refiera al hecho de que en un red neuronal alimentada hacia adelante el error retropropagado normalmente decrece (o incrementa) en función de la distancia desde la capa final.

Además del problema de desvanecimiento, el error del gradiente puede ser inestable en redes neuronales profundas, sufriendo cambios abruptos o explosiones (divergencias), en donde el gradiente crece exponencialmente a medida que se propaga hacia atrás. Este problema se conce como problema del gradiente explosivo:

El problema de los gradientes que se desvanecen es un problema particular en las redes neuronales recurrentes, ya que la actualización de la red implica “desenrrollar” la red, creando una red muy profunda que requiere la actualización de los pesos. Una red neuronal recurrente modesta puede tener entre 200 a 400 pasos de tiempo de entrada, lo que resulta conceptualmente en una red muy profunda.

El problema de los gradientes que se desvanecen se puede manifestar en un perceptrón multicapas mediante una rata baja de la mejora del modelo durante el entrenamiento, y, quizas, en una convergencia temprana, es decir, el entrenamiento continuo no implica una mejora de la precisión del modelo. La inspección de los cambios de los pesos durante el entrenamiento, nos debería llevar a mayores cambios (que implica un mayor aprendizaje) en las capas cercanas a la salida  y cambios menores en las capas para cada paso de tiempo de entrada. 

Existen varias técnicas que pueden ser usadas para reducir el impacto de los gradientes que se desvanecen en el caso de redes neuronales alimentadas hacia adelante, los más notables de ellos son :

-	Los esquema de inicialización de pesos alternantes.
-	El uso de funciones de activación alternativas.

Se han estudiado diferentes aproximaciones para el entrenamiento de redes neuronales profundas (tanto para redes alimentadas hacia adelante como las recurrentes), en un esfuerzo para abordar los gradientes que se desvanecen, como el pre-entrenamiento, una mejora en el escalado aleatorio inicial, las mejoras en los métodos de optimización, el estudio de arquitecturas específicas, las inicializaciones ortogonales, etc.

En esta sección echaremos una mirada más cerca al uso de funciones de activación y esquemás alternativos de inicialización de los pesos, para poder entrenar redes neuronales más profundas. 


## Problema inducido por la funciones de activación:

Como vimos en la clase anterior, el algoritmo de retropropagación requiere de las derivadas de las funciones de activación, lo que implica que algunas de ellas favorecen el desvanecimento del gradiente.

Veamos que ocurre con las derivadas de las funciones de activación usadas en las capas ocultas.

### Función sigmoide:

La función sigmoide se define de la forma 

$$S(x)=\frac{1}{1+e^{-x}}$$

en donde la gráfica tiene la siguiente forma:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1/(1+np.exp(-x))

x = np.linspace(-10,10,100)

plt.plot(x,sigmoid(x),'-')

La cual tiene un rango definido en el intervalo abierto $(0,1)$.

La derivada de esta función, podemos calcularla como:

In [ ]:
from sympy import *

x = Symbol('x')
f = 1/(1+exp(-x))
f.diff(x)

In [ ]:
def sigmoid_deff(x):
  return np.exp(-x)/(1 + np.exp(-x))**2

x = np.linspace(-10,10,100)
plt.plot(x,sigmoid_deff(x),'-')

Veamos además, que el mapero de la función de activación **sigmoide** genera una concentración en las colas de la función, en donde la derivada se hace cero:

In [ ]:
import tensorflow as tf
from tensorflow import keras

x=np.linspace(-5,5,100)

y_sigmoid = keras.activations.sigmoid(x)

y_sigmoid_diff = np.gradient(y_sigmoid)

plt.figure(figsize=(14,5))

plt.subplot(121)
plt.plot(x,y_sigmoid,'.')
plt.title('Función de Activación sigmoide')
plt.subplot(122)
plt.plot(x,y_sigmoid_diff,'.')
plt.title('Derivada de la función de Activación sigmoide')

**Problemas de la función sigmoide:**

**1) Desvanecimiento del gradiente:** Vemos que la derivada de esta función está muy cerca a cero en todas partes, excepto en el ancho representado por la gaussian. Lo anterior representa el caso a favor del desvanecimiento del gradiente. Por este motivo, la función tangente hiperbólica fue usada en su lugar.

**2) Función no centrada en cero:** La función sigmoide representa una función no centrada en cero, generando valores positivos después de que la activación tenga lugar. Sin entrar en detalles, este hecho ocaciona que el gradiente de los pesos los convierta a todos en positivos o negativos, lo que dificulta la actualización del gradiente, comportándose de forma erratica. A este problema se le conoce como el [problema de funciones no centradas en cero](https://stats.stackexchange.com/questions/237169/why-are-non-zero-centered-activation-functions-a-problem-in-backpropagation).

#### Función de activación Tanh:

La función de activación tangente hiperbólica se define como:

$$\text{Tanh}(x)=\frac{e^x-​e^{​‑x}}{e^x+​e^{‑x}}$$

cuya gráfica es:

In [ ]:
x = np.linspace(-7,7,100)

plt.plot(x,np.tanh(x),'-')

Cuyo rango es el intervalo abierto $(-1,1)$.

La derivada de esta función es:

In [ ]:
x = Symbol('x')
f = (exp(x)-exp(-x))/(exp(x)+exp(-x))
f.diff(x)

In [ ]:
def tanh_diff(x):
  return (-np.exp(x)+np.exp(-x))*(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))**2 + 1

x = np.linspace(-7,7,100)

plt.plot(x,tanh_diff(x),'-')

Veamos además, que el mapeo de la función de activación la función **tanh** genera una concentración de puntos en las colas de la función, en donde su derivada se hace cero:

In [ ]:
# función tanh
x=np.linspace(-5,5,100)

y_tanh = keras.activations.tanh(x)

#Derivada
y_tanh_diff = np.gradient(y_tanh)

# Gráfica de la función de activación y su derivada:
plt.figure(figsize=(14,5))

plt.subplot(121)
plt.plot(x,y_tanh,'.')
plt.title('Función de Activación Tanh')
plt.subplot(122)
plt.plot(x,y_tanh_diff,'.')
plt.title('Derivada de la función de Activación Tanh')

**Problemas de la función de activación tanh**

**1) Desvanecimiento del gradiente:** A pesar de se ha demostrado que el desempeño de la función de activación es mejor, dado su rango $(-1,1)$, la derivada de esta función es muy cercana a cero en todo el dominio, excepto dentro de un intervalo pequeño centrado en el origen y determinado por el ancho de la gaussiana. Problema similar al de la función sigmoide.


### Función de activación ReLU:

Con el fin de corregir el problema introduccido por el desvanecimiento del gradiente al usar las funciones de activación **sigmoide** y **tangente hiperbólica**, se introduce la función **ReLU**.

Vemos las características de esta función y su derivada:

In [ ]:
x=np.linspace(-50,50,101)

# Función de activación ReLU
y_relu = keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)

# Derivada de la función de activación ReLU
y_relu_diff = np.gradient(y_relu)

# Gráfica de la función de activación ReLU y su derivada:
plt.figure(figsize=(14,5))

plt.subplot(121)
plt.plot(x,y_relu,'.')
plt.title('Función de Activación ReLU')
plt.subplot(122)
plt.plot(x,y_relu_diff,'.')
plt.title('Derivada de la función de Activación ReLU')


**Problemas de la función de activación ReLU**

**1) Dying ReLU:**  El problema principal de la función de activación ReLU está asociaado con el hecho de asignar un valor cero a todos los valores negativos provenientes del ajuste lineal. Decimos que una neurona se encuentra muerta cuando el ajuste arroja valores negativos. Como vemos en la gráfica de la derivada ReLU, ésta es cero en los valores negativos de la función. Una vez que la neurona se vuelve "negativa", es muy poco probable que se recupere, y se convertirá en una neurona muerta. El problema de las neuronas muertas es que corresponenden a unidades inservibles, incapaces de contribuir en la clasificación. A este problema se le conoce con el nombre en inglés de [Dying ReLU](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7). Una solución a este problema es aplicar una función de activación conocida como LeakyReLU, la cual puede ser consultada en la referencia anterior y no abordaremos en este módulo.

A pesar de este problema, veremos que la función de activación resuelve el problema del desvanecimiento del gradiente en redes neuronales profundas. 

### Función de activación SeLU:

A continuación, hablaremos brevemente de la funciones SELU (Scaled Exponencial Linear Units), Las cuales aparecen por primera vez en el siguiente
[artículo](https://arxiv.org/pdf/1706.02515.pdf).

Para una explicación detallada de la función de activación SELU, pueden consultar la siguiente 
[referencia](https://towardsdatascience.com/gentle-introduction-to-selus-b19943068cd9).

La forma de la función SELU y su derivada es la siguiente:


In [ ]:
x=np.linspace(-20,20,100)

y_selu = keras.activations.selu(x)

y_selu_diff = np.gradient(y_selu)

plt.figure(figsize=(14,5))

plt.subplot(121)
plt.plot(x,y_selu,'.')
plt.title('Función de Activación ReLU')
plt.subplot(122)
plt.plot(x,y_selu_diff,'.')
plt.title('Derivada de la función de Activación ReLU')

De esta  gráfica podemos extraer dos características principales de la función de activación SELU:

1. El lado derecho de la función, es similar a la función de activación ReLU.

2. El lado izquierdo, parace aproximarse a un gradiente zero, que es justo lo que queriamos evitar. No obstante la función SELU realiza un proceso aparte del control del gradiente conocído como **normalización**.

Respecto a la normalización podemos tener tres casos:

1. Que la normalización tenga lugar a la entrada de la red, por ejemplo cuando se reescala un intervalo de valor de píxeles que va de 0-255 a uno entre 0-1. Este tipo de normalización es una buena practica en ML.

2. En el caso de  NN, existe un normalización relevante conocida como **batch normalization**, la cual tiene lugar entre las capas de la red, transformandose las salidas de tal manera que el valor medio es cero y la desviación estándar sea uno. La principal ventaja de este procedimiento es que limita los valores y hace menos probable que ocurran valores extremales donde se hace cero la derivada. 

3. También en el caso de NN, la normalización puede tener lugar **internamente**, como en el caso de la función SELU. La idea principal es que cada capa preserva el valor medio y la varianza de la capa anterior, evitándose de esta forma valores extremales. 

**Problemas de la función SELU**

1) Función reciente que necesita ser estudiada.

2) Computacionalmente costosa.

# Redes Neuronales Profundas

En las clases previas introdujimos las redes neuronales artificiales y vimos algunos ejemplos de cómo entrenarlas. Pero estas eran redes muy poco profundas, con solo unas pocas capas ocultas. ¿Qué sucede si necesita abordar un problema muy complejo, como detectar cientos de tipos de objetos en imágenes de alta resolución? (Como ejemplo, considere el DIUx xView 2018 Detection Challenge http://xviewdataset.org/). Para estos casos, es posible que se necesite entrenar un DNN mucho más profunda, tal vez con 10 capas o más, cada una con cientos de neuronas, conectadas por cientos de miles de conexiones. Aparecerán algunos problemas cuando intentes entrenar redes neuronales realmente profundas, algunos de ellos son:

* Vanishing gradients y exploding gradients que afecta las redes neuronales profundas y hace que las capas inferiores sean muy difíciles de entrenar.

* Es posible que no tengas suficientes datos de entrenamiento para una red tan grande, o puede ser demasiado costoso etiquetarlos.

* El entrenamiento puede ser extremadamente lento.

* un modelo con millones de parámetros correría el riesgo de generar overfitting, especialmente si no hay suficientes instancias de entrenamiento o si son demasiado ruidosas.

En ésta lección veremos algunas técnicas para resolver algunos de estos problemas.



## Ejemplificación del desvanecimiento del gradiente:

A continuación haremos uso de un ejemplo de juguete para ilustrar el problema del desvanecimiento del gradiente dos gupos de puntos (clase 1 y clase 0) con una distribución circular.

Recordemos como crear un dataset a partir de la clase `make_circles` del módulo `datasets` de la librería `sklean`:

In [ ]:
from sklearn.datasets import make_circles
import numpy as np
import matplotlib.pyplot as plt

# Generacíon de dos grupos de círculos.

X, y= make_circles(n_samples = 1000, noise=0.1, random_state=1)

# selección de las clases para graficarlas 
plt.scatter(X[y == 0, 0], X[y == 0, 1], label=str(0),c='k') # extacción puntos según clase (0)
plt.scatter(X[y == 1, 0], X[y == 1, 1], label=str(1),c='g') # extacción puntos según clase (1)

plt.legend()
plt.show()

**Explicación del código:**

En la primeta parte buscamos todos indices con etiquetas`y==0` y etiquetas `y==1`. Para eso usamos la función de `numpy` `where`. Para el caso en que `y == 0`, tenemos que:

In [ ]:
np.where(y == 0)

Si este array (tupla en realidad), se la pasamos en la indexación `X`, obtenemos las instancias (puntos `(X[idx,0],X[idx,1])`), tenemos los valores de las instancias para cada clase (0 y 1):

In [ ]:
X[np.where(y==0)]

In [ ]:
# Preprocesado de los datos:
from sklearn.preprocessing import MinMaxScaler

# scale input data to [-1,1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)

In [ ]:
# División de los datos en entrenamiento y evaluación
n_train = 500
train_X, test_X = X[:n_train, :], X[n_train:, :]
train_y, test_y = y[:n_train], y[n_train:]

### Modelo del perceptrón multicapas para el problema de dos círculos:

Empecemos por ilustrar el caso en que temos un ajuste adecuado del modelo de de una red neuronal poco profunda consituida por una capa de entrada, con una función de activación `tanh` y una capa de salida `sigmoid`. Este modelo, no debe sufrir de problemas de desvanecimiento/explosión del gradiente:

In [ ]:
from tensorflow import keras

keras.backend.clear_session()

# Definición del modelo poco profundo
modelA= keras.models.Sequential()
init = keras.initializers.RandomUniform(minval=0, maxval=1)
modelA.add(keras.layers.Dense(5, input_dim=2, activation='tanh', kernel_initializer=init))
modelA.add(keras.layers.Dense(1, activation='sigmoid', kernel_initializer=init))

# Compilación del modelo
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9)
modelA.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])
modelA.summary()

In [ ]:

# Ajuste del modelo:
history = modelA.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=500)

In [ ]:

# evaluate the model
_, train_acc = modelA.evaluate(train_X, train_y, verbose=0)
_, test_acc = modelA.evaluate(test_X, test_y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
y_pred=modelA.predict_classes(X)

In [ ]:

plt.scatter(X[y == 0, 0], X[y == 0, 1], label=str(0)) # extacción puntos según clase (0)
plt.scatter(X[y == 1, 0], X[y == 1, 1], label=str(1)) # extacción puntos según clase (1)

plt.legend()
plt.show()

In [ ]:
y_pred=y_pred.reshape((-1))
plt.scatter(X[y_pred == 0, 0], X[y_pred == 0, 1], label=str(0)) # extacción puntos según clase (0)
plt.scatter(X[y_pred == 1, 0], X[y_pred == 1, 1], label=str(1)) # extacción puntos según clase (1)

plt.legend()
plt.show()

### Creación de un modelo más profundo con fución de activación tanh:

Veamos ahora cómo un modelo de una red neuronal más profunda, genera un modelo menos predictivo devido al problema del desvanecimiento del gradiente. La red consta de una capa de entrada con un a función de activación `tanh`, y 4 capas ocultas, también definidas con una función de activación `tanh`, y una capa de salida con una función de activación `sigmoid`.



In [ ]:
keras.backend.clear_session()

# define model
init = keras.initializers.RandomUniform(minval=0, maxval=1)
modelB = keras.models.Sequential()
modelB.add(keras.layers.Dense(5, input_dim=2, activation='tanh', kernel_initializer=init))
modelB.add(keras.layers.Dense(5, activation='tanh', kernel_initializer=init))
modelB.add(keras.layers.Dense(5, activation='tanh', kernel_initializer=init))
modelB.add(keras.layers.Dense(5, activation='tanh', kernel_initializer=init))
modelB.add(keras.layers.Dense(5, activation='tanh', kernel_initializer=init))
modelB.add(keras.layers.Dense(1, activation='sigmoid', kernel_initializer=init))
# compile model
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9)
modelB.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit model
history = modelB.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=500, verbose=0)
# evaluate the model
_, train_acc = modelB.evaluate(train_X, train_y, verbose=0)
_, test_acc = modelB.evaluate(test_X, test_y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
y_pred=modelB.predict_classes(X)

In [ ]:
y_pred=y_pred.reshape((-1))
plt.scatter(X[y_pred == 0, 0], X[y_pred == 0, 1], label=str(0)) # extacción puntos según clase (0)
plt.scatter(X[y_pred == 1, 0], X[y_pred == 1, 1], label=str(1)) # extacción puntos según clase (1)

plt.legend()
plt.show()

### Creación de un modelo más profundo con fución de activación ReLU:

Ilustremos ahora cómo, con el simple hecho de cambiar las función de activación de la capa de entrada y las capas ocultas del anterior modelo por una función de activación `ReLU`, se mejora el problema del desvanecimiento del gradiente:

In [ ]:
keras.backend.clear_session()

# define model
modelC = keras.models.Sequential()
modelC.add(keras.layers.Dense(5, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
modelC.add(keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'))
modelC.add(keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'))
modelC.add(keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'))
modelC.add(keras.layers.Dense(5, activation='relu', kernel_initializer='he_uniform'))
modelC.add(keras.layers.Dense(1, activation='sigmoid'))
# compile model
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9)
modelC.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit model
history = modelC.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=500, verbose=0)
# evaluate the model
_, train_acc = modelC.evaluate(train_X, train_y, verbose=0)
_, test_acc = modelC.evaluate(test_X, test_y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

In [ ]:
y_pred=modelC.predict_classes(X)
y_pred=y_pred.reshape((-1))
plt.scatter(X[y_pred == 0, 0], X[y_pred == 0, 1], label=str(0)) # extacción puntos según clase (0)
plt.scatter(X[y_pred == 1, 0], X[y_pred == 1, 1], label=str(1)) # extacción puntos según clase (1)

plt.legend()
plt.show()

Finalmente, veamos que las funciones de activación `SeLU`, también  ayudan a resolver el problema del desvanecimiento/explosión del gradiente:

In [ ]:
keras.backend.clear_session()

# define model
modelE = keras.models.Sequential()
modelE.add(keras.layers.Dense(5, input_dim=2, activation='selu', kernel_initializer='lecun_normal'))
modelE.add(keras.layers.Dense(5, activation='selu', kernel_initializer='lecun_normal'))
modelE.add(keras.layers.Dense(5, activation='selu', kernel_initializer='lecun_normal'))
modelE.add(keras.layers.Dense(5, activation='selu', kernel_initializer='lecun_normal'))
modelE.add(keras.layers.Dense(5, activation='selu', kernel_initializer='lecun_normal'))
modelE.add(keras.layers.Dense(1, activation='sigmoid'))
# compile model
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9)
modelE.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit model
history = modelE.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=500, verbose=0)
# evaluate the model
_, train_acc = modelE.evaluate(train_X, train_y, verbose=0)
_, test_acc = modelE.evaluate(test_X, test_y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

# Recapitulación del problema del desvanecimiento/explosión del gradiente.

Cuando se realiza la propagación hacia atrás, los gradientes a menudo se hacen cada vez más pequeños a medida que el algoritmo avanza hacia las capas inferiores. Como resultado, el algoritmo de Descenso de degradado deja los pesos de conexión de la capa inferior prácticamente sin cambios, y el entrenamiento nunca converge en una buena solución. Esto es lo que se conoce como Vanishing Gradients Problems. En algunos otros casos, puede suceder lo contrario: los gradientes pueden crecer más y más, por lo que muchas capas obtienen actualizaciones de peso increíblemente grandes y el algoritmo diverge. Esto es lo que se conoce como Exploding Gradients Problems.  


Aunque este comportamiento se ha observado empíricamente durante bastante tiempo (fue una de las razones por las que las redes neuronales profundas fueron abandonadas durante mucho tiempo), apenas alrededor de 2010 se logró un progreso significativo en su comprensión. Xavier Glorot y Yoshua Bengio descubrieron que el problema estaba en la combinación de la popular función de activación sigmoid (y tambien la hyperbolica aunque en menor medida) y la técnica de inicialización de pesos que era más popular en ese momento (la inicialización aleatoria usando una distribución normal con una media de 0 y un estándar desviación de 1).

<p><img alt="Colaboratory logo" height="300px" src="https://i.imgur.com/Dbt0lIL.png" align="center" hspace="10px" vspace="0px"></p> 


# Inicialización de Glorot and He

Glorot y Bengio proponen una forma de aliviar significativamente este problema. Argumentan que necesitamos la varianza de la
Las salidas de cada capa deben ser iguales a la varianza de sus entradas, y también necesitamos que los gradientes tengan la misma varianza antes y después de fluir a través de una capa en la dirección inversa. En realidad, no es posible garantizar ambos a menos que la capa tenga el mismo número de entradas y neuronas (que en general no es cierto), pero propusieron una buena solución que ha demostrado funcionar muy bien en la práctica: los pesos de cada capa (para el caso de la funcion de activacion sigmoid) deben inicializarse aleatoriamente como (Esta es la inicialización de Glorot):

Distribución normal con media 0 y varianza
\begin{equation}
\sigma^2 = \frac{1}{fan_{avg}}
\end{equation}

O una distribución uniforme entre $−r$ and $+r$,
\begin{equation}
r = \sqrt{\frac{3}{fan_{avg}}} = \sqrt{3\sigma^2}
\end{equation}


donde 


* $fan_{avg} = \frac{1}{2} (fan_{in} + fan_{out})$


* $fan_{in}$ es el numero de neuronas de entrada 

* $fan_{out}$ es el numero de neuronas de salida 


La estrategia de inicialización para la función de activación ReLUy sus variantes es (Esta es la inicialización de He) :

Distribución normal con media 0 y varianza
\begin{equation}
\sigma^2 = \frac{2}{fan_{avg}}
\end{equation}

O una distribución uniforme entre $−r$ and $+r$,
\begin{equation}
r = \sqrt{\frac{6}{fan_{avg}}} = \sqrt{3\sigma^2}
\end{equation}

podemos resumir entonces la forma mas comun de inicialzar los pesos en la sigueinte tabla 

<p><img alt="Colaboratory logo" height="200px" src="https://i.imgur.com/hBQ26T3.png" align="center" hspace="10px" vspace="0px"></p> 






Por defecto, Keras usa la inicialización de Glorot con una distribución uniforme, pero esto puede ser cambiado. Veamos un ejemplo de como hacerlo

importemos algunas librerias que seran de utilidad

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()
from sklearn.datasets import make_circles

Resolveremos el problema de las dos espirales, para esto comencemos con crear las espirales

In [ ]:
def twospirals(n_points, noise=0.5):
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), 
            np.hstack((np.zeros(n_points),np.ones(n_points))))

In [ ]:
X, y =  twospirals(1000, noise=0.8)
plt.figure(figsize=(7,7))
plt.scatter(X[:,0], X[:,1], c=y, cmap='winter')

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import datetime, os
keras.backend.clear_session()

Dividamos los datos en un set de entrenamiento y otro de testeo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

escalemos los datos

In [ ]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, cmap='winter')

In [ ]:
X_train.shape

In [ ]:
X_test.shape

procedamos a construir nuestro modelo y a ilustrar como incializar los pesos de diferentes formas dependiendo la funcion de activacion

In [ ]:
model = keras.models.Sequential([                                
                                keras.layers.Dense(40, activation='relu', kernel_initializer='he_normal',input_shape= (2,)),
                                keras.layers.Dense(12, activation='tanh',kernel_initializer='glorot_normal'),
                                keras.layers.Dense(40, activation='relu',kernel_initializer='he_uniform'),
                                keras.layers.Dense(12, activation='tanh',kernel_initializer='glorot_normal'),
                                keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform')
                                ])
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train, epochs=200 ,verbose=0)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(7,7))

In [ ]:
model.evaluate(X_test,y_test, verbose=0)

In [ ]:
y_fit =  model.predict_classes(X_test)

In [ ]:
fig , ax = plt.subplots(1,2, figsize=(8,6))
ax[0].scatter(X_test[:,0], X_test[:,1], c=y_test, cmap='winter')
ax[0].set_title('true_model')
ax[1].scatter(X_test[:,0], X_test[:,1], c=y_fit[:,0], cmap='winter')
ax[1].set_title('predicted_model')

Por defecto, Keras implementa la inicialización He con una distribución uniforme basada en $fan_{in}$. Si desea la inicialización de He con una distribución uniforme pero basada en $fan_{avg}$, puede usar el inicializador VarianceScaling de esta manera:

In [ ]:
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

mas informacion de la versatilidad a la hora de inicializar los pesos puede ser encontrado en https://keras.io/initializers/

# Batch Normalization

Hemos visto que es una buena práctica normalizar los datos de entrada. Este procedimiento también ayuda en las capas intermedias para reducir el problema de los gradientes que se desvanecen. En estas capas se conoce como Batch Normalization y se busca reescalar los datos de manera que cambie su media y desviación estándar.

Un perceptrón multicapa sigue la siguiente estructura:

$$X^l=f^l(X^{l-1}A^l+B^l)$$

De aquí vemos que hay 3 formas de atacar al problema de los gradientes que se desvancen.


*   Funciones de Activación $f^l$ que no se saturan (ReLU,SELU)
*   Inicialización de $A^l$ y $B^l$ (Glorot, He)
*   Normalización de $X^l$ (Batch Normalization)

En sesiones anteriores se discutieron las 2 primeras alternativas y en este caso se analizará el Batch Normalization.

El primer paso consta en normalizar los datos:

$$X^l\to \frac{X^l-\mu^l}{\sigma^l}$$

Dónde $\mu^l$ y $\sigma^l$ son el promedio y la desviación estándar por lote de datos en la capa $l$. De esta manera, los datos en la capa $l$ quedaran normalizados con media nula y desviación estándar unitaria. Durante el entrenamiento deben acumularse los valores de $\mu^l$ y $\sigma^l$ de todos los lotes de datos para usarlos a la hora de realizar inferencia.  De aquí salen 2 parámetros por cada nodo (promedio y desviación) que se deben tener en cuenta. Sin embargo, estos parámetros no se optimizan usando gradiente descendente sino que se calculan por lotes.

No es suficiente estandarizar los datos con media nula y varianza unitaria. Es posible que la red neuronal aprenda una normalización más eficiente. Es por esto que los datos se reescalan nuevamente de la siguiente manera:

$$X^l\to \gamma \frac{X^l-\mu^l}{\sigma^l}+\beta$$

Dónde $\beta$ y $\gamma$ serán el nuevo promedio y desviación estándar de los datos. Estos parámetros si debe aprenderlos la red neuronal durante el entrenamiento, por lo que tenemos 2 parámetros entrenables adicionales por cada nodo.

Implementemos el Batch Normalization en Keras para el caso del dataset de Iris

In [ ]:
from sklearn.datasets import load_iris
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization #Importar BatchNorm
from tensorflow.keras.utils import to_categorical

Se importan los datos, se normalizan y se hace un one-hot encoding de los labels

In [ ]:
data=load_iris()
X,y=data.data,data.target
X-=X.mean(axis=0)
X/=X.std(axis=0)
y=to_categorical(y)

Se construye una red neuronal con 4 capas y Batch Normalization después de la segunda capa

In [ ]:
modelo=Sequential()
modelo.add(Dense(4,activation='relu',input_shape=(4,)))
modelo.add(Dense(4,activation='relu'))
modelo.add(BatchNormalization())
modelo.add(Dense(4,activation='relu'))
modelo.add(Dense(3,activation='softmax'))
modelo.compile('sgd',loss='categorical_crossentropy',metrics=['accuracy'])
modelo.summary()

En la capa de Batch Normalization hay 16 parámetros pero sólo 8 de esos son entrenables. Los 8 parámetros entrenables son los cuatro $\gamma$ y cuatro $\beta$ por cada neurona. Los 8 parámetros **no**-entrenables son los promedios $\mu^l$ y las desviaciones estándar $\sigma^l$ por cada nodo. Por último se entrena el modelo:

In [ ]:
modelo.fit(x=X,y=y, batch_size=10,epochs=500)

# Gradient Clipping

Aparte de los gradientes que se desvanecen, es posible que al hacer Backpropagation, los gradientes se acumulen de tal manera que resulten valores computacionalmente grandes. Este problema se conoce como Gradientes que Explotan y para lidiar con él debe limitarse el valor del gradiente. Esta solución es mucho más sencilla que los gradientes que explotan y sólo implica darle estabilidad numérica al gradiente. En keras hay 2 formas de hacerlo:


*   Clip Norm: Se reescala todo el gradiente para que su norma no supere ciero umbral ($U$). Esta opción tiene la ventaja de conservar la dirección del gradiente:
$$\nabla J \to \nabla J \frac{U}{|\nabla J|}$$
*   Clip Value: Se reescalan únicamente los valores que superen cierto umbral ($U$).
$$(\nabla J)_i \to (\nabla J)_i \frac{U}{|\nabla J|_i}$$



El Gradient Clipping se implementa en el optimizador. Debido a esto debemos importar el optimizador explícitamente

In [ ]:
from sklearn.datasets import load_iris
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD #Importar el optimizador
from tensorflow.keras.utils import to_categorical

Nuevamente se importan, se estandarizan y se codifican los datos

In [ ]:
data=load_iris()
X,y=data.data,data.target
X-=X.mean(axis=0)
X/=X.std(axis=0)
y=to_categorical(y)

Se construye el modelo con 3 capas

In [ ]:
modelo=Sequential()
modelo.add(Dense(4,activation='sigmoid',input_shape=(4,)))
modelo.add(Dense(4,activation='sigmoid'))
modelo.add(Dense(4,activation='sigmoid'))
modelo.add(Dense(3,activation='softmax'))

El optimizador hace el gradient clipping con el argumento "clipnorm" o "clipvalue" dependiendo del modo que se elija:

In [ ]:
optim=SGD(clipnorm=1) #Se cortan los gradientes estableciendo un umbral máximo para la norma
modelo.compile(optim,loss='categorical_crossentropy',metrics=['accuracy'])
modelo.summary()

Por último, se entrena la red:

In [ ]:
modelo.fit(x=X,y=y, batch_size=10,epochs=500)